In [1]:
%pip install spacy > /dev/null
%pip install scikit-learn > /dev/null
%pip install pandas > /dev/null


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# For spanish language
from spacy.lang.es.stop_words import STOP_WORDS as STOP_WORDS_ES

In [3]:
products = pd.read_csv("data/content_data.csv")
products

,Unnamed: 0,Title,Description,Keywords
0,Articulo1,Aprendiendo Python,"Este libro cubre los fundamentos de Python, in...",Python programación desarrollo
1,Articulo2,Introducción a Machine Learning,Un curso sobre los conceptos básicos de Machin...,Machine Learning IA algoritmos
2,Articulo3,Guía de Data Science,Explora técnicas de Data Science y cómo trabaj...,Data Science análisis datos
3,Articulo4,Aprende programación en R,Un tutorial para comenzar a programar en R par...,R estadística programación
4,Articulo5,Deep Learning avanzado,Profundiza en técnicas avanzadas de Deep Learn...,Deep Learning redes neuronales


In [4]:
products["Content"] = products["Description"] + " " + products["Keywords"]
products

,Unnamed: 0,Title,Description,Keywords,Content
0,Articulo1,Aprendiendo Python,"Este libro cubre los fundamentos de Python, in...",Python programación desarrollo,"Este libro cubre los fundamentos de Python, in..."
1,Articulo2,Introducción a Machine Learning,Un curso sobre los conceptos básicos de Machin...,Machine Learning IA algoritmos,Un curso sobre los conceptos básicos de Machin...
2,Articulo3,Guía de Data Science,Explora técnicas de Data Science y cómo trabaj...,Data Science análisis datos,Explora técnicas de Data Science y cómo trabaj...
3,Articulo4,Aprende programación en R,Un tutorial para comenzar a programar en R par...,R estadística programación,Un tutorial para comenzar a programar en R par...
4,Articulo5,Deep Learning avanzado,Profundiza en técnicas avanzadas de Deep Learn...,Deep Learning redes neuronales,Profundiza en técnicas avanzadas de Deep Learn...


In [5]:
tfidf = TfidfVectorizer(stop_words=list(STOP_WORDS_ES))
tfidf_matrix = tfidf.fit_transform(products["Content"])

In [6]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.        , 0.09953524, 0.07392761, 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.17655677],
       [0.09953524, 0.        , 1.        , 0.06953418, 0.04202227],
       [0.07392761, 0.        , 0.06953418, 1.        , 0.        ],
       [0.        , 0.17655677, 0.04202227, 0.        , 1.        ]])

In [7]:
indexes = pd.Series(products.index, index=products["Title"]).drop_duplicates()
indexes

Title
Aprendiendo Python                 0
Introducción a Machine Learning    1
Guía de Data Science               2
Aprende programación en R          3
Deep Learning avanzado             4
dtype: int64

In [8]:
def get_recommendations(idxs, cosine_sim):
    final_recomendations = None

    for idx in idxs:
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Top 5, Excluding the first element (itself)
        sim_scores = sim_scores[1:6]
        product_indexes = [i[0] for i in sim_scores]

        scores = [i[1] for i in sim_scores]
        recomendations = indexes.iloc[product_indexes].index

        # Add scores to the recomendations
        recomendations = pd.DataFrame(recomendations)
        recomendations["Score"] = scores

        if final_recomendations is None:
            final_recomendations = recomendations
            continue

        final_recomendations = pd.concat([final_recomendations, recomendations])
        final_recomendations = (
            final_recomendations.groupby("Title")
            .mean()
            .sort_values("Score", ascending=False)
            .reset_index()
        )
    return final_recomendations

In [9]:
recommendations = get_recommendations([0], cosine_sim)
print("Artículos recomendados:")
print(recommendations)

Artículos recomendados:
                             Title     Score
0             Guía de Data Science  0.099535
1        Aprende programación en R  0.073928
2  Introducción a Machine Learning  0.000000
3           Deep Learning avanzado  0.000000


In [10]:
recommendations = get_recommendations([1], cosine_sim)
print("Recommended articles:")
print(recommendations)

Artículos recomendados:
                       Title     Score
0     Deep Learning avanzado  0.176557
1         Aprendiendo Python  0.000000
2       Guía de Data Science  0.000000
3  Aprende programación en R  0.000000


In [11]:
recommendations = get_recommendations([0, 1], cosine_sim)
print("Recommended articles:")
print(recommendations)

Artículos recomendados:
                             Title     Score
0           Deep Learning avanzado  0.088278
1             Guía de Data Science  0.049768
2        Aprende programación en R  0.036964
3               Aprendiendo Python  0.000000
4  Introducción a Machine Learning  0.000000
